In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # Skip connection
        out = F.relu(out)
        return out

In [3]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

# For ResNet-18: 2 blocks per layer (total conv layers: 1 initial + 2*4*2 = 17, but counted as 18 including FC? Standard naming)
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:14<00:00, 12.2MB/s]


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = ResNet18().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)  # Adjust LR over epochs

def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(f'Epoch {epoch}: Loss: {train_loss / (batch_idx + 1):.3f} | Acc: {100. * correct / total:.2f}%')

def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f'Test Epoch {epoch}: Loss: {test_loss / (batch_idx + 1):.3f} | Acc: {100. * correct / total:.2f}%')

# Run training loop
for epoch in range(1, 21):
    train(epoch)
    test(epoch)
    scheduler.step()

Epoch 1: Loss: 1.974 | Acc: 30.09%
Test Epoch 1: Loss: 1.631 | Acc: 39.27%
Epoch 2: Loss: 1.497 | Acc: 45.12%
Test Epoch 2: Loss: 1.337 | Acc: 50.32%
Epoch 3: Loss: 1.241 | Acc: 55.15%
Test Epoch 3: Loss: 1.119 | Acc: 58.67%
Epoch 4: Loss: 0.994 | Acc: 64.79%
Test Epoch 4: Loss: 1.240 | Acc: 56.45%
Epoch 5: Loss: 0.814 | Acc: 71.38%
Test Epoch 5: Loss: 0.874 | Acc: 69.60%
Epoch 6: Loss: 0.696 | Acc: 75.61%
Test Epoch 6: Loss: 0.714 | Acc: 75.84%
Epoch 7: Loss: 0.615 | Acc: 78.72%
Test Epoch 7: Loss: 0.723 | Acc: 74.93%
Epoch 8: Loss: 0.550 | Acc: 81.08%
Test Epoch 8: Loss: 0.654 | Acc: 77.73%
Epoch 9: Loss: 0.504 | Acc: 82.53%
Test Epoch 9: Loss: 0.601 | Acc: 79.16%
Epoch 10: Loss: 0.454 | Acc: 84.21%
Test Epoch 10: Loss: 0.586 | Acc: 80.28%
Epoch 11: Loss: 0.410 | Acc: 85.92%
Test Epoch 11: Loss: 0.575 | Acc: 80.34%
Epoch 12: Loss: 0.374 | Acc: 87.10%
Test Epoch 12: Loss: 0.535 | Acc: 81.65%
Epoch 13: Loss: 0.327 | Acc: 88.70%
Test Epoch 13: Loss: 0.491 | Acc: 84.08%
Epoch 14: Loss: 0